# Drug Overdose Modeling

In [36]:
import numpy as np
import pandas as pd
import scipy as sp
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import patsy 
import math

pd.set_option('display.max_columns', 100)
%matplotlib inline

## Import the feature DataFrame

In [37]:
# Read the DF created from the feature_selection notebook
counties = pd.read_pickle('counties_df.pkl')

In [38]:
# Make the column names standardized for use in patsy formulas
counties.columns = [x.replace(' ', '_').replace('-', '_').replace('%', 'Percent').replace('.', '_').replace('(', '').replace(')', '').replace('<','lt') for x in list(counties.columns)]
# Patsy also doesn't like column names begining with a number
counties = counties.rename(columns={'20th_Percentile_Income': 'Percentile_Income_20th', '80th_Percentile_Income': 'Percentile_Income_80th'})

## Train a model on the appropriate features

In [39]:
counties

,State,County,Age_Adjusted_Mortality,Age_Adjusted_Mortality_Black,Age_Adjusted_Mortality_Hispanic,Age_Adjusted_Mortality_White,Child_Mortality_Rate,Child_Mortality_Rate_Black,Child_Mortality_Rate_White,Infant_Mortality_Rate,Percent_Frequent_Physical_Distress,Percent_Frequent_Mental_Distress,Percent_Diabetic,HIV_Prevalence_Rate,Percent_Food_Insecure,Percent_Limited_Access,Drug_Overdose_Mortality_Rate,MV_Mortality_Rate,Percent_Insufficient_Sleep,Percent_Uninsured,Percent_Uninsured_1,Costs,Other_PCP_Rate,Percent_Disconnected_Youth,Household_Income,Household_income_Black,Household_income_Hispanic,Household_income_White,Percent_Free_or_Reduced_Lunch,Segregation_index,Segregation_Index,Homicide_Rate,Firearm_Fatalities_Rate,Population,Percent_lt_18,Percent_65_and_over,Percent_African_American,Percent_American_Indian/Alaskan_Native,Percent_Asian,Percent_Native_Hawaiian/Other_Pacific_Islander,Percent_Hispanic,Percent_Non_Hispanic_White,Percent_Not_Proficient_in_English,Percent_Female,Percent_Rural,State_ranked,County_ranked,Years_of_Potential_Life_Lost_Rate,Years_of_Potential_Life_Lost_Rate_Black,Years_of_Potential_Life_Lost_Rate_Hispanic,...,Percent_LBW_Hispanic,Percent_LBW_White,Percent_Smokers,Percent_Obese,Food_Environment_Index,Percent_Physically_Inactive,Percent_With_Access,Percent_Excessive_Drinking,Percent_Alcohol_Impaired,Chlamydia_Rate,Teen_Birth_Rate,Teen_Birth_Rate_Black,Teen_Birth_Rate_Hispanic,Teen_Birth_Rate_White,Percent_Uninsured_ranked,PCP_Rate,Dentist_Rate,MHP_Rate,Preventable_Hosp__Rate,Percent_Receiving_HbA1c,Percent_Receiving_HbA1c_Black,Percent_Receiving_HbA1c_White,Percent_Mammography,Percent_Mammography_Black,Percent_Mammography_White,Cohort_Size,Graduation_Rate,Population_ranked,Percent_Some_College,Percent_Unemployed,Percent_Children_in_Poverty,Percent_Children_in_Poverty_Black,Percent_Children_in_Poverty_Hispanic,Percent_Children_in_Poverty_White,Percentile_Income_80th,Percentile_Income_20th,Income_Ratio,Percent_Single_Parent_Households,Association_Rate,Violent_Crime_Rate,Injury_Death_Rate,Average_Daily_PM2_5,Presence_of_violation,Percent_Severe_Housing_Problems,Percent_Drive_Alone,Percent_Drive_Alone_Black,Percent_Drive_Alone_Hispanic,Percent_Drive_Alone_White,Percent_Long_Commute___Drives_Alone,Urban
FIPS,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1001,Alabama,Autauga,449.6,531.700000,292.315327,444.900000,56.186903,95.921022,48.240956,7.718502,12.824434,12.638357,12.4,225.500000,13.9,11.991125,10.231226,20.167754,35.905406,12.193092,2.595313,10674.34,28.87253,14.570000,54487.0,32173.000000,72411.000000,58556.000000,48.475388,26.347048,24.000912,3.619853,16.972533,55416,24.408113,14.728598,19.048650,0.485419,1.140465,0.101054,2.688754,75.000000,0.832739,51.288437,42.002162,Alabama,Autauga,9409.294766,11806.223285,5968.227286,...,7.260143,7.415319,19.124658,36.4,7.1,30.9,71.120192,16.878015,26.785714,341.2,27.996758,31.533043,38.461538,27.153838,9.387712,45.16957,32.48159,12.63173,81.85,83.966245,74.107143,85.809683,62.693157,58.333333,63.209877,706.0,90.000000,14315,61.942019,5.271161,19.3,28.072153,16.699411,14.400244,101520.0,23134.0,4.388346,24.471409,13.912226,265.699747,72.584664,10.8,0.0,15.217927,86.750062,82.494851,83.676458,82.539683,40.8,True
1003,Alabama,Baldwin,351.3,517.800000,111.100000,349.300000,47.688777,87.008289,44.472180,5.545407,12.622002,12.822558,11.1,163.900000,13.0,5.424427,18.452504,15.281185,33.305868,14.719861,3.273439,9537.12,43.63190,15.410000,56460.0,26818.000000,41908.000000,54184.000000,45.795480,43.969941,30.674915,2.705282,14.718943,208563,21.734440,19.708194,9.042831,0.770990,1.074975,0.059454,4.416411,83.202198,0.818809,51.457833,42.279099,Alabama,Baldwin,7467.596664,10335.702691,2827.348216,...,4.672131,8.019411,16.795485,29.3,7.9,24.5,68.518915,16.714969,30.379747,338.8,31.965138,38.553804,57.609217,29.475711,11.524397,72.65266,49.38556,89.18169,46.30,83.212237,82.432432,83.259019,65.965233,65.753425,65.972551,

In [40]:
formula = "Drug_Overdose_Mortality_Rate ~ 1 + Percent_Unemployed + Household_Income + Graduation_Rate + Percent_Rural + HIV_Prevalence_Rate + Percent_Frequent_Mental_Distress + Percent_Excessive_Drinking"
y, X = patsy.dmatrices(formula, counties, return_type='matrix') 
model = sm.OLS(y, X)

In [41]:
results = model.fit()

In [42]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                 
========================================================================================
Dep. Variable:     Drug_Overdose_Mortality_Rate   R-squared:                       0.169
Model:                                      OLS   Adj. R-squared:                  0.166
Method:                           Least Squares   F-statistic:                     47.91
Date:                          Wed, 12 Dec 2018   Prob (F-statistic):           3.67e-62
Time:                                  00:21:14   Log-Likelihood:                -6081.7
No. Observations:                          1655   AIC:                         1.218e+04
Df Residuals:                              1647   BIC:                         1.222e+04
Df Model:                                     7                                         
Covariance Type:                      nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                          -19.2633      6.156     -3.129      0.002     -31.337      -7.189
Percent_Unemployed                   0.8768      0.174      5.036      0.000       0.535       1.218
Household_Income                     0.0001    2.9e-05      3.467      0.001    4.37e-05       0.000
Graduation_Rate                      0.0669      0.035      1.924      0.055      -0.001       0.135
Percent_Rural                        0.0172      0.011      1.597      0.110      -0.004       0.038
HIV_Prevalence_Rate                 -0.0027      0.001     -2.109      0.035      -0.005      -0.000
Percent_Frequent_Mental_Distress     2.2124      0.274      8.084      0.000       1.676       2.749
Percent_Excessive_Drinking          -0.2322      0.099     -2.347      0.019      -0.426      -0.038
==============================================================================
Omnibus:                      535.045   Durbin-Watson:                   1.631
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2169.778
Skew:                           1.518   Prob(JB):                         0.00
Kurtosis:                       7.717   Cond. No.                     1.42e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.42e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## See what SciKitLearn thinks are the best features to use

In [43]:
from sklearn.feature_selection import SelectKBest, f_regression
X = counties.copy().drop(columns=['Drug_Overdose_Mortality_Rate'])
X = X.select_dtypes(exclude='object')
y = counties['Drug_Overdose_Mortality_Rate']
kbest = SelectKBest(f_regression, k=20).fit(X, y)

In [44]:
new_features = []
mask = kbest.get_support()

for boolean, feature in zip(mask, list(counties.columns.values)):
    if boolean:
        new_features.append(feature)

In [45]:
new_features

['State',
 'Age_Adjusted_Mortality',
 'Age_Adjusted_Mortality_Black',
 'Child_Mortality_Rate',
 'Child_Mortality_Rate_Black',
 'Child_Mortality_Rate_White',
 'Infant_Mortality_Rate',
 'Percent_Frequent_Physical_Distress',
 'Percent_Uninsured_1',
 'Segregation_index',
 'Percent_Not_Proficient_in_English',
 'State_ranked',
 'Years_of_Potential_Life_Lost_Rate',
 'Years_of_Potential_Life_Lost_Rate_Black',
 'Physically_Unhealthy_Days',
 'Mentally_Unhealthy_Days',
 'Percent_Smokers',
 'Teen_Birth_Rate_White',
 'Percent_Some_College',
 'Percentile_Income_20th']

## Compare our intuitive features choices against the K-best

In [46]:
formula = "Drug_Overdose_Mortality_Rate ~ 1"
for feature in new_features:
    formula += ' + ' + feature
formula

'Drug_Overdose_Mortality_Rate ~ 1 + State + Age_Adjusted_Mortality + Age_Adjusted_Mortality_Black + Child_Mortality_Rate + Child_Mortality_Rate_Black + Child_Mortality_Rate_White + Infant_Mortality_Rate + Percent_Frequent_Physical_Distress + Percent_Uninsured_1 + Segregation_index + Percent_Not_Proficient_in_English + State_ranked + Years_of_Potential_Life_Lost_Rate + Years_of_Potential_Life_Lost_Rate_Black + Physically_Unhealthy_Days + Mentally_Unhealthy_Days + Percent_Smokers + Teen_Birth_Rate_White + Percent_Some_College + Percentile_Income_20th'

In [47]:
y, X = patsy.dmatrices(formula, counties, return_type='matrix') 
model = sm.OLS(y, X)

In [48]:
results = model.fit()

In [49]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                 
========================================================================================
Dep. Variable:     Drug_Overdose_Mortality_Rate   R-squared:                       0.574
Model:                                      OLS   Adj. R-squared:                  0.556
Method:                           Least Squares   F-statistic:                     31.47
Date:                          Wed, 12 Dec 2018   Prob (F-statistic):          9.44e-244
Time:                                  00:21:15   Log-Likelihood:                -5528.4
No. Observations:                          1655   AIC:                         1.119e+04
Df Residuals:                              1586   BIC:                         1.157e+04
Df Model:                                    68                                         
Covariance Type:                      nonrobust                                         
===========================================================================================================
                                              coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------
Intercept                                 -19.2794      6.375     -3.024      0.003     -31.783      -6.776
State[T.Alaska]                             4.8945      1.924      2.543      0.011       1.120       8.669
State[T.Arizona]                            4.4482      1.278      3.481      0.001       1.942       6.955
State[T.Arkansas]                          -0.6883      0.892     -0.772      0.440      -2.438       1.061
State[T.California]                         4.1262      0.864      4.776      0.000       2.432       5.821
State[T.Colorado]                           5.6241      0.992      5.672      0.000       3.679       7.569
State[T.Connecticut]                        7.7483      1.421      5.453      0.000       4.961      10.535
State[T.Delaware]                           7.0096      2.131      3.290      0.001       2.830      11.189
State[T.District of Columbia]               3.2218      3.595      0.896      0.370      -3.830      10.273
State[T.Florida]                            4.4709      0.868      5.153      0.000       2.769       6.173
State[T.Georgia]                            1.8464      0.855      2.158      0.031       0.168       3.524
State[T.Hawaii]                             3.1186      1.912      1.631      0.103      -0.632       6.869
State[T.Idaho]                              3.3369      1.305      2.557      0.011       0.778       5.896
State[T.Illinois]                           4.0441      0.860      4.703      0.000       2.357       5.731
State[T.Indiana]                            4.1535      0.863      4.814      0.000       2.461       5.846
State[T.Iowa]                               2.6893      1.124      2.393      0.017       0.485       4.893
State[T.Kansas]                             2.3015      1.181      1.948      0.052      -0.016       4.619
State[T.Kentucky]                           8.1844      0.761     10.759      0.000       6.692       9.676
State[T.Louisiana]                          1.9388      0.841      2.306      0.021       0.290       3.588
State[T.Maine]                              5.6268      1.150      4.894      0.000       3.371       7.882
State[T.Maryland]                           6.5947      0.985      6.697      0.000       4.663       8.526
State[T.Massachusetts]                      7.6709      1.205      6.366      0.000       5.307      10.034
State[T.Michigan]                           4.8377      0.784      6.167      0.000       3.299       6.376
State[T.Minnesota]                          3.4729      1.041      3.337      0.001       1.432       5.514
State[T.Mississippi]                       -1.3768      0

## Building a Logistic Regression model to predict Drug Overdose Mortality Rate Percentile

#### In this section, we will break counties into top 25% and bottom 25% of Drug Overdose Mortality Rate and build a logistic regression model to predict which group a county might be in.



In [50]:
counties

,State,County,Age_Adjusted_Mortality,Age_Adjusted_Mortality_Black,Age_Adjusted_Mortality_Hispanic,Age_Adjusted_Mortality_White,Child_Mortality_Rate,Child_Mortality_Rate_Black,Child_Mortality_Rate_White,Infant_Mortality_Rate,Percent_Frequent_Physical_Distress,Percent_Frequent_Mental_Distress,Percent_Diabetic,HIV_Prevalence_Rate,Percent_Food_Insecure,Percent_Limited_Access,Drug_Overdose_Mortality_Rate,MV_Mortality_Rate,Percent_Insufficient_Sleep,Percent_Uninsured,Percent_Uninsured_1,Costs,Other_PCP_Rate,Percent_Disconnected_Youth,Household_Income,Household_income_Black,Household_income_Hispanic,Household_income_White,Percent_Free_or_Reduced_Lunch,Segregation_index,Segregation_Index,Homicide_Rate,Firearm_Fatalities_Rate,Population,Percent_lt_18,Percent_65_and_over,Percent_African_American,Percent_American_Indian/Alaskan_Native,Percent_Asian,Percent_Native_Hawaiian/Other_Pacific_Islander,Percent_Hispanic,Percent_Non_Hispanic_White,Percent_Not_Proficient_in_English,Percent_Female,Percent_Rural,State_ranked,County_ranked,Years_of_Potential_Life_Lost_Rate,Years_of_Potential_Life_Lost_Rate_Black,Years_of_Potential_Life_Lost_Rate_Hispanic,...,Percent_LBW_Hispanic,Percent_LBW_White,Percent_Smokers,Percent_Obese,Food_Environment_Index,Percent_Physically_Inactive,Percent_With_Access,Percent_Excessive_Drinking,Percent_Alcohol_Impaired,Chlamydia_Rate,Teen_Birth_Rate,Teen_Birth_Rate_Black,Teen_Birth_Rate_Hispanic,Teen_Birth_Rate_White,Percent_Uninsured_ranked,PCP_Rate,Dentist_Rate,MHP_Rate,Preventable_Hosp__Rate,Percent_Receiving_HbA1c,Percent_Receiving_HbA1c_Black,Percent_Receiving_HbA1c_White,Percent_Mammography,Percent_Mammography_Black,Percent_Mammography_White,Cohort_Size,Graduation_Rate,Population_ranked,Percent_Some_College,Percent_Unemployed,Percent_Children_in_Poverty,Percent_Children_in_Poverty_Black,Percent_Children_in_Poverty_Hispanic,Percent_Children_in_Poverty_White,Percentile_Income_80th,Percentile_Income_20th,Income_Ratio,Percent_Single_Parent_Households,Association_Rate,Violent_Crime_Rate,Injury_Death_Rate,Average_Daily_PM2_5,Presence_of_violation,Percent_Severe_Housing_Problems,Percent_Drive_Alone,Percent_Drive_Alone_Black,Percent_Drive_Alone_Hispanic,Percent_Drive_Alone_White,Percent_Long_Commute___Drives_Alone,Urban
FIPS,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1001,Alabama,Autauga,449.6,531.700000,292.315327,444.900000,56.186903,95.921022,48.240956,7.718502,12.824434,12.638357,12.4,225.500000,13.9,11.991125,10.231226,20.167754,35.905406,12.193092,2.595313,10674.34,28.87253,14.570000,54487.0,32173.000000,72411.000000,58556.000000,48.475388,26.347048,24.000912,3.619853,16.972533,55416,24.408113,14.728598,19.048650,0.485419,1.140465,0.101054,2.688754,75.000000,0.832739,51.288437,42.002162,Alabama,Autauga,9409.294766,11806.223285,5968.227286,...,7.260143,7.415319,19.124658,36.4,7.1,30.9,71.120192,16.878015,26.785714,341.2,27.996758,31.533043,38.461538,27.153838,9.387712,45.16957,32.48159,12.63173,81.85,83.966245,74.107143,85.809683,62.693157,58.333333,63.209877,706.0,90.000000,14315,61.942019,5.271161,19.3,28.072153,16.699411,14.400244,101520.0,23134.0,4.388346,24.471409,13.912226,265.699747,72.584664,10.8,0.0,15.217927,86.750062,82.494851,83.676458,82.539683,40.8,True
1003,Alabama,Baldwin,351.3,517.800000,111.100000,349.300000,47.688777,87.008289,44.472180,5.545407,12.622002,12.822558,11.1,163.900000,13.0,5.424427,18.452504,15.281185,33.305868,14.719861,3.273439,9537.12,43.63190,15.410000,56460.0,26818.000000,41908.000000,54184.000000,45.795480,43.969941,30.674915,2.705282,14.718943,208563,21.734440,19.708194,9.042831,0.770990,1.074975,0.059454,4.416411,83.202198,0.818809,51.457833,42.279099,Alabama,Baldwin,7467.596664,10335.702691,2827.348216,...,4.672131,8.019411,16.795485,29.3,7.9,24.5,68.518915,16.714969,30.379747,338.8,31.965138,38.553804,57.609217,29.475711,11.524397,72.65266,49.38556,89.18169,46.30,83.212237,82.432432,83.259019,65.965233,65.753425,65.972551,

In [51]:
#Calulte cutoffs for top and bottom percentiles
top_25_cutoff =  float(pd.DataFrame(counties['Drug_Overdose_Mortality_Rate']).quantile(q=0.75, axis=0, numeric_only=True, interpolation='linear'))
bottom_25_cutoff = float(pd.DataFrame(counties['Drug_Overdose_Mortality_Rate']).quantile(q=0.25, axis=0, numeric_only=True, interpolation='linear'))


In [82]:
#Drop anything in middle 50%
counties = counties.drop(counties[(counties['Drug_Overdose_Mortality_Rate'] < top_25_cutoff) &  (counties['Drug_Overdose_Mortality_Rate'] > bottom_25_cutoff)].index)
counties.head()


,State,County,Age_Adjusted_Mortality,Age_Adjusted_Mortality_Black,Age_Adjusted_Mortality_Hispanic,Age_Adjusted_Mortality_White,Child_Mortality_Rate,Child_Mortality_Rate_Black,Child_Mortality_Rate_White,Infant_Mortality_Rate,Percent_Frequent_Physical_Distress,Percent_Frequent_Mental_Distress,Percent_Diabetic,HIV_Prevalence_Rate,Percent_Food_Insecure,Percent_Limited_Access,Drug_Overdose_Mortality_Rate,MV_Mortality_Rate,Percent_Insufficient_Sleep,Percent_Uninsured,Percent_Uninsured_1,Costs,Other_PCP_Rate,Percent_Disconnected_Youth,Household_Income,Household_income_Black,Household_income_Hispanic,Household_income_White,Percent_Free_or_Reduced_Lunch,Segregation_index,Segregation_Index,Homicide_Rate,Firearm_Fatalities_Rate,Population,Percent_lt_18,Percent_65_and_over,Percent_African_American,Percent_American_Indian/Alaskan_Native,Percent_Asian,Percent_Native_Hawaiian/Other_Pacific_Islander,Percent_Hispanic,Percent_Non_Hispanic_White,Percent_Not_Proficient_in_English,Percent_Female,Percent_Rural,State_ranked,County_ranked,Years_of_Potential_Life_Lost_Rate,Years_of_Potential_Life_Lost_Rate_Black,Years_of_Potential_Life_Lost_Rate_Hispanic,...,Percent_LBW_White,Percent_Smokers,Percent_Obese,Food_Environment_Index,Percent_Physically_Inactive,Percent_With_Access,Percent_Excessive_Drinking,Percent_Alcohol_Impaired,Chlamydia_Rate,Teen_Birth_Rate,Teen_Birth_Rate_Black,Teen_Birth_Rate_Hispanic,Teen_Birth_Rate_White,Percent_Uninsured_ranked,PCP_Rate,Dentist_Rate,MHP_Rate,Preventable_Hosp__Rate,Percent_Receiving_HbA1c,Percent_Receiving_HbA1c_Black,Percent_Receiving_HbA1c_White,Percent_Mammography,Percent_Mammography_Black,Percent_Mammography_White,Cohort_Size,Graduation_Rate,Population_ranked,Percent_Some_College,Percent_Unemployed,Percent_Children_in_Poverty,Percent_Children_in_Poverty_Black,Percent_Children_in_Poverty_Hispanic,Percent_Children_in_Poverty_White,Percentile_Income_80th,Percentile_Income_20th,Income_Ratio,Percent_Single_Parent_Households,Association_Rate,Violent_Crime_Rate,Injury_Death_Rate,Average_Daily_PM2_5,Presence_of_violation,Percent_Severe_Housing_Problems,Percent_Drive_Alone,Percent_Drive_Alone_Black,Percent_Drive_Alone_Hispanic,Percent_Drive_Alone_White,Percent_Long_Commute___Drives_Alone,Urban,top_25
FIPS,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1001,Alabama,Autauga,449.6,531.7,292.315327,444.9,56.186903,95.921022,48.240956,7.718502,12.824434,12.638357,12.4,225.5,13.9,11.991125,10.231226,20.167754,35.905406,12.193092,2.595313,10674.34,28.87253,14.57,54487.0,32173.0,72411.0,58556.0,48.475388,26.347048,24.000912,3.619853,16.972533,55416,24.408113,14.728598,19.048650,0.485419,1.140465,0.101054,2.688754,75.000000,0.832739,51.288437,42.002162,Alabama,Autauga,9409.294766,11806.223285,5968.227286,...,7.415319,19.124658,36.4,7.1,30.9,71.120192,16.878015,26.785714,341.2,27.996758,31.533043,38.461538,27.153838,9.387712,45.16957,32.48159,12.63173,81.85,83.966245,74.107143,85.809683,62.693157,58.333333,63.209877,706.0,90.000000,14315,61.942019,5.271161,19.3,28.072153,16.699411,14.400244,101520.0,23134.0,4.388346,24.471409,13.912226,265.699747,72.584664,10.8,0.0,15.217927,86.750062,82.494851,83.676458,82.539683,40.8,True,0
1021,Alabama,Chilton,516.3,615.2,256.348989,522.1,56.483879,105.666951,61.207661,6.894791,14.445380,14.260704,14.1,99.5,13.5,1.460531,12.138224,22.140966,35.174569,19.470843,4.762770,10515.38,9.10312,19.58,44188.0,27375.0,28081.0,45494.0,61.087434,36.022367,30.416380,6.512049,15.939158,43941,24.039052,16.055620,9.929223,0.514326,0.475638,0.175235,7.603377,80.560297,2.446886,50.831797,86.744724,Alabama,Chilton,11099.683529,16051.458138,5249.976050,...,8.253752,19.673275,36.0,8.2,29.8,46.044039,15.064313,20.253165,307.3,51.259961,41.629761,65.128901,51.742919,15.232994,18.20540,25.03357,38.68824,65.11,89.059081,88.888889,89.073634,64.000000,60.869565,64.259928,517.0,86.000000,11708,39.118551,5.617919,26.3,49.869452,56.843680,25.272703,84217.0,17180.0,4.9020

In [83]:
# Create binary feature for top 25% vs bottom 25%

counties['top_25'] = counties['Drug_Overdose_Mortality_Rate'] >= top_25_cutoff
counties['top_25'] = counties['top_25']*1

In [84]:
counties[['top_25','Drug_Overdose_Mortality_Rate']].head()

,top_25,Drug_Overdose_Mortality_Rate
FIPS,,
1001,0,10.231226
1021,0,12.138224
1031,0,7.825441
1033,0,8.583007
1039,0,9.716714


In [85]:
#Use formula from previous model and add in our original features of interest
formula_logit = 'top_25 ~ 1 + Age_Adjusted_Mortality + Age_Adjusted_Mortality_Black + Child_Mortality_Rate + Child_Mortality_Rate_Black + Child_Mortality_Rate_White + Infant_Mortality_Rate + Percent_Frequent_Physical_Distress + Percent_Uninsured_1 + Segregation_index + Percent_Not_Proficient_in_English + Years_of_Potential_Life_Lost_Rate + Years_of_Potential_Life_Lost_Rate_Black + Physically_Unhealthy_Days + Mentally_Unhealthy_Days + Percent_Smokers + Teen_Birth_Rate_White + Percent_Some_College + Percentile_Income_20th + Percent_Unemployed + Household_Income + Graduation_Rate + Percent_Rural + HIV_Prevalence_Rate + Percent_Frequent_Mental_Distress + Percent_Excessive_Drinking'
y, X = patsy.dmatrices(formula_logit, counties, return_type='matrix') 
model_logit = sm.Logit(y,X)
results_logit = model_logit.fit()

Optimization terminated successfully.
         Current function value: 0.373546
         Iterations 7


In [86]:
results_logit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 top_25   No. Observations:                  828
Model:                          Logit   Df Residuals:                      802
Method:                           MLE   Df Model:                           25
Date:                Wed, 12 Dec 2018   Pseudo R-squ.:                  0.4611
Time:                        00:42:18   Log-Likelihood:                -309.30
converged:                       True   LL-Null:                       -573.93
                                        LLR p-value:                 6.545e-96
===========================================================================================================
                                              coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------
Intercept                                 -14.5474      3.563     -4.083      0.000     -21.531      -7.564
Age_Adjusted_Mortality                      0.0135      0.005      2.490      0.013       0.003       0.024
Age_Adjusted_Mortality_Black                0.0015      0.002      0.966      0.334      -0.002       0.005
Child_Mortality_Rate                       -0.0443      0.011     -4.018      0.000      -0.066      -0.023
Child_Mortality_Rate_Black                  0.0262      0.006      4.077      0.000       0.014       0.039
Child_Mortality_Rate_White                  0.0485      0.011      4.217      0.000       0.026       0.071
Infant_Mortality_Rate                       0.2121      0.088      2.411      0.016       0.040       0.385
Percent_Frequent_Physical_Distress         -1.3801      0.501     -2.753      0.006      -2.363      -0.398
Percent_Uninsured_1                        -0.1084      0.055     -1.985      0.047      -0.215      -0.001
Segregation_index                           0.0321      0.008      3.919      0.000       0.016       0.048
Percent_Not_Proficient_in_English          -0.0082      0.078     -0.106      0.916      -0.161       0.144
Years_of_Potential_Life_Lost_Rate           0.0008      0.000      3.418      0.001       0.000       0.001
Years_of_Potential_Life_Lost_Rate_Black    -0.0002   7.39e-05     -2.632      0.008      -0.000   -4.97e-05
Physically_Unhealthy_Days                   2.6282      1.377      1.909      0.056      -0.070       5.326
Mentally_Unhealthy_Days                     4.5303      1.032      4.391      0.000       2.508       6.552
Percent_Smokers                             0.0024      0.062      0.038      0.970      -0.119       0.124
Teen_Birth_Rate_White                      -0.0850      0.016     -5.289      0.000      -0.117      -0.054
Percent_Some_College                       -0.0558      0.021     -2.624      0.009      -0.097      -0.014
Percentile_Income_20th                  -6.938e-05   5.89e-05     -1.178      0.239      -0.000    4.61e-05
Percent_Unemployed                          0.0449      0.099      0.453      0.651      -0.149       0.239
Household_Income                         8.332e-05   2.91e-05      2.861      0.004    2.62e-05       0.000
Graduation_Rate                             0.0094      0.016      0.576      0.565      -0.023       0.041
Percent_Rural                              -0.0306      0.007     -4.320      0.000      -0.045      -0.017
HIV_Prevalence_Rate                        -0.0014      0.001     -2.204      0.028      -0.003      -0.000
Percent_Frequent_Mental_Distress           -0.8984      0.440     -2.042      0.041      -1.761      -0.036
Percent_Excessive_Drinking                  0.0408      0.050      0.823      0.411      -0.056       0.138
===========================================================================================================
"

### Looking at Logistic Regression Coefficients


In [87]:
# Convert from log odds to odds ratios
for ix,x in enumerate(np.exp(results_logit.params)): 
    print (x,X.design_info.column_names[ix])

4.809944668826064e-07 Intercept
1.013624135385344 Age_Adjusted_Mortality
1.0015422954903095 Age_Adjusted_Mortality_Black
0.956624762690044 Child_Mortality_Rate
1.026524256528914 Child_Mortality_Rate_Black
1.049683256656971 Child_Mortality_Rate_White
1.236273999101653 Infant_Mortality_Rate
0.2515625417886249 Percent_Frequent_Physical_Distress
0.8973127695914501 Percent_Uninsured_1
1.0326258334807938 Segregation_index
0.991812776874542 Percent_Not_Proficient_in_English
1.0007719570211537 Years_of_Potential_Life_Lost_Rate
0.999805539997919 Years_of_Potential_Life_Lost_Rate_Black
13.848668305761125 Physically_Unhealthy_Days
92.78408136692597 Mentally_Unhealthy_Days
1.0023662706717684 Percent_Smokers
0.9184914048974326 Teen_Birth_Rate_White
0.9457353095837092 Percent_Some_College
0.9999306250772376 Percentile_Income_20th
1.0458985252896376 Percent_Unemployed
1.0000833202543888 Household_Income
1.0094045018217432 Graduation_Rate
0.9698157087129744 Percent_Rural
0.9986174267410463 HIV_Prevale

In [88]:
# Convert confidence intervals to odds ratios as well
for ix,x in enumerate(np.exp(results_logit.conf_int())): 
    print (x,X.design_info.column_names[ix])

[4.46035079e-10 5.18693905e-04] Intercept
[1.00288315 1.02448016] Age_Adjusted_Mortality
[0.99841425 1.00468014] Age_Adjusted_Mortality_Black
[0.9361549  0.97754222] Child_Mortality_Rate
[1.01368755 1.03952352] Child_Mortality_Rate_Black
[1.02628968 1.07361008] Child_Mortality_Rate_White
[1.04046951 1.46892665] Infant_Mortality_Rate
[0.09418079 0.67193862] Percent_Frequent_Physical_Distress
[0.80625607 0.9986532 ] Percent_Uninsured_1
[1.01617702 1.04934091] Segregation_index
[0.8514144  1.15536287] Percent_Not_Proficient_in_English
[1.00032927 1.00121484] Years_of_Potential_Life_Lost_Rate
[0.99966076 0.99995035] Years_of_Potential_Life_Lost_Rate_Black
[  0.93235757 205.69963659] Physically_Unhealthy_Days
[ 12.28112827 700.9849231 ] Mentally_Unhealthy_Days
[0.88763276 1.13192998] Percent_Smokers
[0.89000481 0.94788978] Teen_Birth_Rate_White
[0.90713805 0.98597482] Percent_Some_College
[0.99981519 1.00004607] Percentile_Income_20th
[0.86133974 1.27000262] Percent_Unemployed
[1.00002623 1